# Homework 4

## FINM 35700 - Spring 2024

### UChicago Financial Mathematics

### Due Date: 2024-04-30

* Alex Popovici
* alex.popovici@uchicago.edu

This homework relies on multiple files (from previous weeks):

- the bond symbology file `bond_symbology`, 
- the "on-the-run" treasuries data file `govt_on_the_run`,
- the bond market data file `bond_market_prices_eod`,
- the CDS data file `cds_market_data_eod`. 
- the SOFR OIS Swap symbology file `sofr_swap_symbology`,
- the SOFR OIS Swap market data file `sofr_swaps_market_data_eod`.



In [3]:
# import tools from previous homeworks
from credit_market_tools import *

# Use static calculation/valuation date of 2024-04-19, matching data available in the market prices EOD file
calc_date = ql.Date(19, 4, 2024)
ql.Settings.instance().evaluationDate = calc_date

# Calculation/valuation date as pd datetime
as_of_date = pd.to_datetime('2024-04-19')

datetime.date(2024, 4, 19)

-----------------------------------------------------------
# Problem 1: Pricing risky bonds in the hazard rate model
## This is building upon
- Homework 2 "Problem 2: US Treasury yield curve calibration (On-The-Runs)",
- Homework 3 "Problem 3: US SOFR swap curve calibration" and
- Homework 3 "Problem 4: CDS Hazard Rate calibration".

## a. Prepare the market data
### Load the symbology + market data dataframes. Calibrate the following curves as of 2024-04-19:
- the "on-the-run" US Treasury curve,
- the US SOFR curve and 
- the IBM CDS hazard rate curve (on the top of SOFR discount curve).


In [7]:
# tsy_yield_curve calibration
govt_combined_otr = []    # TODO: Follow Homework 2 Problem 2 and populate the US Treasury On-The-Run symbology + market data frame !!!
tsy_yield_curve = calibrate_yield_curve_from_frame(calc_date, govt_combined_otr, 'midPrice')
tsy_yield_curve_handle = ql.YieldTermStructureHandle(tsy_yield_curve)


# sofr_yield_curve calibration
sofr_combined = []    # TODO: Follow Homework 3 Problem 3 and populate the SOFR symbology + market data frame !!!
sofr_yield_curve = calibrate_sofr_curve_from_frame(calc_date, sofr_combined, 'midRate')
sofr_yield_curve_handle = ql.YieldTermStructureHandle(sofr_yield_curve)


# hazard_rate_curve calibrated to IBM CDS par spreads
hazard_rate_curve = []    # TODO: Follow Homework 3 Problem 4 and create the IBM hazard rate curve !!!
default_prob_curve_handle = ql.DefaultProbabilityTermStructureHandle(hazard_rate_curve)

## b. Create the IBM risky bond objects
### Identify the following 3 IBM fixed rate bonds in the symbology table and create the corresponding fixed rate bonds (3 bond objects).

- security = 'IBM 3.3 05/15/26' / figi = 'BBG00P3BLH05'
- security = 'IBM 3.3 01/27/27' / figi = 'BBG00FVNGFP3'
- security = 'IBM 3 1/2 05/15/29' / figi = 'BBG00P3BLH14'

Use the create_bond_from_symbology() function (discussed in from Homework 2, Problem 1b) to create the bonds objects.

List the bond cashflows using the get_bond_cashflows() function.



## c. Compute CDS-implied (intrinsic) prices for the IBM fixd rate bonds

Price the 3 IBM bonds using the CDS-calibrated hazard rate curve for IBM (via RiskyBondEngine).

Display the clean prices and yields for the 3 test bonds. 

You can use the example code below.


In [12]:
# flat_recovery_rate: use market convention of 40% for "Senior Unsecured" Debt
flat_recovery_rate = 0.40

fixed_rate_bond = ql.FixedRateBond()    # TODO: Pick one of the 3 IBM test bonds !!!

# Risky bond engine uses the calibrated CDS hazard rate curve for pricing credit default risk 
risky_bond_engine = ql.RiskyBondEngine(default_prob_curve_handle, flat_recovery_rate, tsy_yield_curve_handle)

fixed_rate_bond.setPricingEngine(risky_bond_engine)

corpBondModelPrice = fixed_rate_bond.cleanPrice()

corpBondModelYield = fixed_rate_bond.bondYield(corpBondModelPrice, ql.Thirty360(ql.Thirty360.USA), ql.Compounded, ql.Semiannual) * 100

print('corpBondCleanPrice', corpBondModelPrice)
print('corpBondYield', corpBondModelYield)


## d. Compute the "intrinsic" vs market price basis for the IBM bonds

Load the market mid prices and yields from the bond market data dataframe as of 2024-04-19. 

Compute and display the basis between the "CDS-implied intrinsic" vs market prices and yields:

- basisPrice = modelPrice - midPrice
- basisYield = modelYield - midYield


Are the CDS intrinsic prices lower or higher than the bond prices observed on the market? What factors could explain the basis?

-----------------------------------------------------------
# Problem 2: Compute scenario sensitivities for risky bonds
## a. Compute scenario IR01s and Durations for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 1. 

Compute the scenario IR01 and Durations using a '-1bp' interest rate shock, as described in Section 6. "Market Data Scenarios" in the QuantLib Basics notebook.

Display the computed scenario IR01 and Durations.

Remember that IR01 = Dirty_Price * Duration.


## b. Compute analytical DV01s and Durations for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 1. 

Compute and display the analytical IR01 and Durations 

Compare the analytic DV01s vs. the scenario IR01s. Are they expected to be similar?

## c. Compute scenario CS01s (credit spread sensitivities) for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 1. 

Apply a '-1bp' (parallel shift) scenario to the IBM CDS Par Spread quotes and calibrate the scenario hazard rate curve. 

Create a new scenario RiskyBondEngine, using the scenario hazard rate curve.

Reprice the risky bonds on the scenario RiskyBondEngine (using the bumped hazard rate curve) to obtain the '-1bp' scenario CS01 (credit spread sensitivities).

Compare the scenario CS01s vs analytic DV01s. Are they expected to be similar?


## d. Compute scenario REC01 (recovery rate sensitivity) for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 1. 

Apply a +1% scenario bump to the IBM recovery rate (bump the flat_recovery_rate parameter by 1%, from 40% to 41%).

Create a new scenario RiskyBondEngine, using the scenario new recovery rate.

Reprice the risky bonds on the scenario RiskyBondEngine (using the bumped recovery rate) to obtain the +1% scenario REC01 (recovery rate sensitivity).


-----------------------------------------------------------
# Problem 3: Perpetual CDS
We are interested in a perpetual CDS contract (infinite maturity) on a face notional of $100, flat interest rate of 4% and coupon of 5% (quarterly payments).

For simplicity, we assuming a flat hazard rate of 1% per annum, a recovery rate of 40%, T+0 settlement and zero accrued.

Use the simple CDS valuation formulas derived in Session 3 as a template.

## a. Compute the fair value of the CDS premium and default legs.


## b. Compute the CDS PV and the CDS Par Spread.

## c. Compute the following CDS risk sensitivities:
- IR01 (PV sensitivity to Interest Rate change of '-1bp')
- HR01 (PV sensitivity to Hazard Rate change of '-1bp')
- REC01 (PV sensitivity to Recovery Rate change of '+1%')

using the scenario method.


## d. At what time T does the (implied) default probability over next 10 years drop to 1%?

\begin{align}
\mathbb{P} \left(\tau \in [T, T+10] \right) = 1/100
\end{align}


-----------------------------------------------------------
# Problem 4: Nelson-Siegel model for smooth hazard rate curves

## This exercise implements tsome of the concepts introduced in Lecture 4, Section 1 "Parametric Hazard Rate Models"

## Follow Section "3. Smooth parametric yield and hazard rate curves: the Nelson-Siegel model" in the QuantLib Advanced notebook
## You can also take a look at Dr. Mark Hendricks Fixed Income notebooks describing the Nelson-Siegel model calibration on US Treasuries (GitHub repo link posted in Canvas).

## a. Prepare the market data as of 2024-04-19
Load the symbology + market data dataframes and create a combined dataframe for for all Verizon (ticker = 'VZ') fixed rate (cpn_type == 'FIXED') bonds with an outstanding amount greater than $100 MM (amt_out > 100).

Sort the dataframe by bond maturity and display the head of the dataframe.

Plot the VZ yields (Y-axis) by TTM (X-axis).

## b. Create the Nelson-Siegel curve shape (4 parameters) and compute the corresponding SSE function.
For a given set of parameters, write a function to compute the SSE "Sum of Squared Errors" penalty function in price space (defined as sum of squared differences between model and market prices for all Verizon fixed-rate bonds).


For each bond, compute the bond DV01, using Section "9. Analytical Duration" in the QuantLib Basics notebook as a template.

Use 1/DV01 as SSE weights, as discussed in Lecture 3. You can ignore the liquidity adjuster for the purpose of this exercise.

## c. Calibrate the Nelson-Siegel model parameters to obtain the smooth Verizon credit curve.

Use the US "on-the-run" Treasury yield curve (already calibrated in Problem 1) for risk-free discounting.

Minimize the SSE (pricing error) function to obtain the optimal/calibrated Nelson-Siegel parameter vector. 

Create the calibrated/smooth credit curve corresponding to the optimal model parameters.

## d. Compute smooth model prices, yields and "edges"

Price all Verizon bonds on the calibrated credit curve and compute the corresponding model yields and edges.

Add following columns to the dataframe and display the head of the results:

| modelPrice | modelYield | edgePrice | edgeYield |
|----------|----------|----------|----------|


## e. Visualize the results of the credit model calibration

Plot the model vs market prices (Y-axis) by maturity (X-axis).

Plot the model vs market yields (Y-axis) by maturity (X-axis).

Plot the edges in yield space (Y-axis) by maturity (X-axis).

What do you think about the quality of the model fit?